In [ ]:
import os
import pickle
import pandas as pd 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models

In [ ]:
df = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
df.head()

In [ ]:
plt.figure(figsize=(18, 10))
df.breed.value_counts().plot(kind ='bar');
plt.show()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
mapping_label = {}
inv_mapping_label = {}
i = 0
for breed in df.breed.unique():
    mapping_label[breed] = i
    inv_mapping_label[i] = breed
    i += 1

print(mapping_label)
print(inv_mapping_label)

In [ ]:
df['breed'] = df.breed.map(mapping_label)
df.head()

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.1, shuffle=True, random_state = 42)
print('Train data shape ', df_train.shape)
print('Validation data shape ', df_val.shape)

In [ ]:
class DogIdentificationDataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.root = root
        self.transform = transform
        self.ids = df.id.values
        self.breed = df.breed.values
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, index):
        ids = self.ids[index]
        target = self.breed[index]
        
        imagePath = os.path.join(self.root, ids + '.jpg')
        image = Image.open(imagePath).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        target = torch.tensor(target, dtype=torch.long)
        
        return image, target
        
        

In [ ]:
def MetricScore(y_true, out):
    prob = F.softmax(out, dim=1)
    pred = torch.argmax(prob, dim=1)
    pred = pred.detach().cpu().numpy()
    y_true = y_true.detach().cpu().numpy()
    score = accuracy_score(y_true, pred)
    return score

In [ ]:
class Net(nn.Module):
    def __init__(self, n_classes, pretrained = True, model ='resnext'):
        super(Net, self).__init__()
        if model == 'resnext':
            self.net = models.resnext50_32x4d(pretrained=pretrained)
            self.net.fc = nn.Linear(2048, n_classes)
        elif model == 'vgg19':
            self.net = models.vgg19(pretrained=pretrained)
            self.net.classifier[6] = nn.Linear(4096, n_classes)
    
    def forward(self, x):
        x = self.net(x)
        return x

In [ ]:
root = '/kaggle/input/dog-breed-identification/train'

trainTransform = T.Compose([
    T.Resize((256, 256)),
    T.RandomCrop((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomPerspective(),
    T.ToTensor(),
    T.Normalize(
        (0.4766, 0.4524, 0.3928),
        (0.2272, 0.2225, 0.2208))
])

valTransform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        (0.4766, 0.4524, 0.3928),
        (0.2272, 0.2225, 0.2208))
])

trainDataset = DogIdentificationDataset(df_train, root, transform=trainTransform)
valDataset = DogIdentificationDataset(df_val, root, transform=valTransform)
trainLoader = DataLoader(trainDataset, batch_size=64, shuffle=True, num_workers=0)
valLoader = DataLoader(valDataset, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
n_classes = df.breed.nunique()
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(n_classes, pretrained=True).to(DEVICE)


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
num_epochs = 100

min_val_loss = np.inf
max_val_score = 0

early_stop = False
n_epochs_stop = 5
epochs_no_improve = 0
last_epoch = 0

train_scores = []
val_scores = []

train_losses = []
val_losses = []

if os.path.exists('./checkpoint.pth'):
    print('Load checkpoint')
    checkpoint = torch.load('./checkpoint.pth')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    min_val_loss = checkpoint['min_val_loss']
    max_val_score = checkpoint['max_val_score']
    last_epoch = checkpoint['epoch']


lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

for epoch in range(last_epoch,num_epochs):
        
    train_loss = 0
    val_loss = 0
    
    train_score = 0
    val_score = 0
    
    model.train()
    pbar = tqdm(trainLoader, total = len(trainLoader))
    for image, target in pbar:
        image = image.to(DEVICE)
        target = target.to(DEVICE)
        
        out = model(image)
        loss = criterion(out, target)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        score = MetricScore(target, out)
        
        train_loss += loss.item()*image.size(0)        
        train_score += score * image.size(0)
        
        pbar.set_postfix({'Train Loss': loss.item(),
                         'Train Score': score})
    with torch.no_grad():
        model.eval()
        pbar = tqdm(valLoader, total=len(valLoader))
        for image, target in pbar:
            image = image.to(DEVICE)
            target = target.to(DEVICE)
            
            out = model(image)
            loss = criterion(out, target)
            
            score = MetricScore(target, out)
            
            val_loss += loss.item()*image.size(0)
            val_score += score*image.size(0)
            
            pbar.set_postfix({'Val Loss': loss.item(),
                             'Val Score': score})
            
    lr_scheduler.step()
    
    train_loss = train_loss/len(trainLoader.dataset)
    val_loss = val_loss/len(valLoader.dataset)
    
    train_score = train_score/len(trainLoader.dataset)
    val_score = val_score/len(valLoader.dataset)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    train_scores.append(train_score)
    val_scores.append(val_score)
    
    print(f'Epoch [{epoch+1}/{num_epochs}] -- Train Loss: {train_loss:.3f} Val Loss: {val_loss:.3f} || Train Score: {train_score*100:.2f}  Val Score: {val_score*100:.2f}')
    
    if min_val_loss > val_loss and max_val_score < val_score:
        min_val_loss = val_loss
        max_val_score = val_score
        checkpoint = {'state_dict': model.state_dict(),
                     'optimizer': optimizer.state_dict(),
                     'min_val_loss': min_val_loss,
                     'max_val_score': max_val_score,
                     'epoch': epoch,
                     'n_classes': n_classes,
                     'mapping_label': mapping_label,
                     'inv_mapping_label':inv_mapping_label}
        
        epochs_no_improve = 0
        torch.save(checkpoint, './checkpoint.pth')
        
    else:
        epochs_no_improve += 1
        
    if epoch > 5 and epochs_no_improve == n_epochs_stop:
        print('Early Stop !!!')
        print('Stopped')
        break
    else:
        continue
        

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(16, 7))
fig.suptitle('Training Plot Dog Identification', fontsize=18)

ax[0].plot(train_losses)
ax[0].plot(val_losses)
ax[0].set_title('Plot Losses', fontsize=14)

ax[1].plot(train_scores)
ax[1].plot(val_scores)
ax[1].set_title('Plot Accuracy Score', fontsize=14)

plt.show()

plt.savefig('./plot.png')

In [ ]:
allDataset = DogIdentificationDataset(df, root, transform=valTransform)
allLoader = DataLoader(allDataset, batch_size=16, shuffle=True, num_workers=0)

checkpoint = torch.load('./checkpoint.pth')
model.load_state_dict(checkpoint['state_dict'])
model.eval()

scores = 0
losses = 0

pbar = tqdm(allLoader, total=len(allLoader))
for image, target in pbar:
    image = image.to(DEVICE)
    target = target.to(DEVICE)
    
    out = model(image)
    loss = criterion(out, target)
    
    score = MetricScore(target, out)
    
    losses += loss.item()*image.size(0)
    scores += score *image.size(0)
    
    pbar.set_postfix({'Loss':loss.item(),
                     'Score':score})

losses = losses / len(allLoader.dataset)
scores = scores / len(allLoader.dataset)

print(f'Loss:{losses:.3f} -- Score:{scores:.3f}')
    